In [1]:
#!pip install gym
#!pip install tensorflow[and-cuda]
#!pip install pygame

In [2]:
# set tf warning level to 2 ....shows errors but not warnings
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
#tf.__version__

In [14]:
from gym import Env
from gym.spaces import Discrete, Box

import pandas as pd
import matplotlib.pyplot as plt


import tensorflow as tf
#tf.config.run_functions_eagerly(True)
#print(tf.config.functions_run_eagerly())
from tensorflow import keras
from tensorflow.keras.models import load_model, Sequential

# for experience replay
from collections import deque
from tensorflow.keras.layers import Dense, Flatten

# tyr legacy adam due to numpy error
from tensorflow.keras.optimizers import Adam
#from tensorflow.keras.optimizers.legacy import Adam

import numpy as np
import random
import os

import pygame


# get component class from custom module
from component_module import Component, plot_correlation_network, plot_component_comparison, getKDEDensity, getDistributionBins, kde_mutual_information, drop_static_columns, performFourierAndLimitHarmonics, scaler_sec_midnight, save_component, load_component, calculate_phase_and_time_difference

# Set the random seed
seed_value = 44
np.random.seed(seed_value)
random.seed(seed_value)

In [15]:
# Check if TensorFlow is built with CUDA support
if tf.config.list_physical_devices('GPU'):
    print("CUDA is available")
    print(f"Device: {tf.config.list_physical_devices('GPU')[0]}")
else:
    print("CUDA is not available")

CUDA is available
Device: PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


I0000 00:00:1727461829.569003    6502 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1727461829.723767    6502 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1727461829.725876    6502 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355


# Create Environment

In [4]:
# ubuntu
df_normalised = pd.read_csv("/home/loz/Documents/GitHub/MSc-Project/SWaT_12_23_sec_minmax.csv", index_col='time_in_seconds')
data_array = df_normalised.to_numpy()
#df_normalised.shape

In [5]:
#df_normalised

In [16]:
data_array[35].max()

1.0

In [17]:
component_pos = {key: idx for idx, key in enumerate(df_normalised.columns)}
#component_pos.items()

In [18]:
#data_array[component_pos['LIT101']]

In [19]:
#df_comp_files

In [20]:
path = '/home/loz/Documents/GitHub/MSc-Project/final_code/component_objs/'
df_comp_files = pd.read_csv(path + 'component_filenames.csv', index_col=0)
components = {}
for component_name, filename in df_comp_files['filename'].items():    
    # put returned component object into dict
    components[component_name] = load_component(filename)
    #print(filename)

Component 'FIT101' has been loaded from /home/loz/Documents/GitHub/MSc-Project/final_code/component_objs/FIT101.
Component 'LIT101' has been loaded from /home/loz/Documents/GitHub/MSc-Project/final_code/component_objs/LIT101.
Component 'MV101' has been loaded from /home/loz/Documents/GitHub/MSc-Project/final_code/component_objs/MV101.
Component 'P101' has been loaded from /home/loz/Documents/GitHub/MSc-Project/final_code/component_objs/P101.
Component 'P102' has been loaded from /home/loz/Documents/GitHub/MSc-Project/final_code/component_objs/P102.
Component 'AIT201' has been loaded from /home/loz/Documents/GitHub/MSc-Project/final_code/component_objs/AIT201.
Component 'AIT202' has been loaded from /home/loz/Documents/GitHub/MSc-Project/final_code/component_objs/AIT202.
Component 'AIT203' has been loaded from /home/loz/Documents/GitHub/MSc-Project/final_code/component_objs/AIT203.
Component 'FIT201' has been loaded from /home/loz/Documents/GitHub/MSc-Project/final_code/component_objs/F

In [21]:
#components['LIT101'].norm_step_size

In [22]:
# test lstm swat simulator
# Load the model from the SavedModel directory
# 20 secs
#swat_lstm = tf.keras.models.load_model('swat_lstm_1_0.keras')

swat_lstm = tf.keras.models.load_model('best_model_20.keras')
model_window = 20

#swat_lstm = tf.keras.models.load_model('swat_lstm_100s.keras')
#model_window = 100



I0000 00:00:1727461840.206590    6502 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1727461840.209715    6502 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1727461840.212199    6502 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1727461840.336925    6502 cuda_executor.cc:1015] successful NUMA node read from SysFS ha

prediction = swat_lstm.predict(reshaped_input)

# Q Learning Table Implementation

In [23]:
# inherit from gym env
class SwatEnv(Env):
    def __init__(self, data_array, window_size=19, episode_length=100):
        print('init reached')
        # 0: both off, 1: MV101 On MV201 Off, 2: MV101 Off MV201 On, 3: both on
        self.action_space = Discrete(4)
        self.episode_length = episode_length
        
        # make sure array is 3 dims for model input
        if len(data_array.shape) == 2:
            # Add a batch dimension to make it (1, time_steps, features)
            data_array = data_array[np.newaxis, :, :]
        
        # swat value array, only position 1 used as lit101 level. 2.d.p 
        self.observation_space = Box(
            # round to 2 d.p. to act as binning values
            low=np.round(np.full((1, 37), 0.00), 2),  # 1x37 array with all elements set to 0.00
            high=np.round(np.full((1, 37), 100.00), 2),  # 1x37 array with all elements set to 100.00
            dtype=np.float32  # Specify float with two decimal places precision
        )
        
        # Initialise data params
        self.data_array = data_array
        self.window_size = window_size 
        
        # use reset to intialise the data window to use 
        self.reset(data_array)      
        
        # state history is 98 time steps preceding
        #self.state_history = model_input
        # set start state as last row in modelinput
        #elf.state = self.state_history [0,-1,:]
        
        # set shower length
        self.episode_length = episode_length
        
        self.lit101_value_record = []
    
    # Override inherited method  
    def sample_observation(self):
        return np.round(self.observation_space.sample(), 2)

        
    def update_state(self, state_history, action_input):
        # Ensure action_input is reshaped to 1x37 array
        action_input = action_input.reshape(1, -1)
        
        # add action to previous states
        model_input = np.concatenate((state_history[:, 1:, :], action_input[:, np.newaxis, :]), axis=1)
        # predict
        
        new_state = swat_lstm.predict(model_input)
        new_state = np.round(new_state, 2)
        
        # prob don't use as new window wanted
        # update history with new state
        #self.state_history = np.concatenate((state_history[:, 1:, :], new_state[:, np.newaxis, :]), axis=1)
        
        # flatten new_state from (1,37) to (37,)
        new_state = new_state.flatten()
        #print(new_state)
        
        return new_state
        
    def step(self, action):
        #print(f'step reached {action}')
        
        # copy current state
        action_input = self.state.copy() 
        
        # MV101 
        if action == 0:
            ## MV101 and MV201 off         
            action_input[component_pos['MV101']] = 0
            action_input[component_pos['MV201']] = 0
            
        elif action == 1:
            # MV101 On, MV201 Off
            action_input[component_pos['MV101']] = 1
            action_input[component_pos['MV201']] = 0
        elif action == 2:
            # MV101 Off, MV201 On
            action_input[component_pos['MV101']] = 0
            action_input[component_pos['MV201']] = 1
        elif action == 3:
            # Both MV101 and MV201 On
            action_input[component_pos['MV101']] = 1
            action_input[component_pos['MV201']] = 1

        
        
        new_state = self.update_state(self.state_history, action_input)
        # test with no actions
        #new_state = self.update_state(self.state_history, self.state.copy())
        
        
        #print(new_state.shape, self.state.shape)
        
        #record lit101 state
        #self.lit101_value_record.append((np.round(new_state[component_pos['LIT101']], 2)))
        # get change in value for plotting
        
        self.lit101_value_record.append(np.round(new_state[component_pos['LIT101']]-self.state[component_pos['LIT101']], 2))

                                     
        # check for increase
        if new_state[component_pos['LIT101']] > self.state[component_pos['LIT101']]:
            reward = new_state[component_pos['LIT101']] - self.state[component_pos['LIT101']]
        elif new_state[component_pos['LIT101']] == self.state[component_pos['LIT101']]:
            reward = -1
        elif new_state[component_pos['LIT101']] < self.state[component_pos['LIT101']]:
            reward = self.state[component_pos['LIT101']] - new_state[component_pos['LIT101']]
            
        #print(f'lit:{self.state[component_pos['LIT101']], new_state[component_pos['LIT101']]} reward:{reward}')
        
        # prob not needed as should be reset to new window
        # update state
        self.state = new_state
        
        done = True
        
        # single episode to allow a new data window
    
        # reduce episode_length
        #self.episode_length = self.episode_length - 1
        
        # Check if episode is done
        #if self.episode_length <= 0:
        #    done = True
        #else:
        #    done = False         
        
        return self.state, reward, self.lit101_value_record, done #, info

    def reset(self, data_array):
        # state history is 98 time steps preceding
        #self.state_history = model_input
        
        # Randomly select new window from data_array for each episode
        start_idx = np.random.randint(0, data_array.shape[1] - self.window_size)
        self.state_history = data_array[:, start_idx:start_idx + self.window_size, :]
        # Set the start state as the last row in the selected window
        self.state = self.state_history[0, -1, :]
        
        # reset episode
        #self.episode_length = episode_length
        # reset value record
        self.lit101_value_record = []
        return self.state
        
    #def render(self):
        # for vis ie pygame
        #pass
    def render(self, screen, font):
        '''
        screen.fill((255, 255, 255))  # Fill the screen with white

        # Draw the temperature meter
        pygame.draw.rect(screen, (0, 0, 255), (100, 150, 50, 200))  # Blue background
        temp_height = (self.state - 0) / 100 * 200  # Scale temp to meter height
        pygame.draw.rect(screen, (255, 0, 0), (100, 350 - temp_height, 50, temp_height))  # Red temperature

        # Display temperature value
        temp_text = font.render(f'Temp: {self.state}°C', True, (0, 0, 0))
        screen.blit(temp_text, (200, 150))

        # Display shower length
        shower_text = font.render(f'Shower Time: {self.shower_length}s', True, (0, 0, 0))
        screen.blit(shower_text, (200, 200))
        '''
        pass

        


In [24]:
bin_size = (1/50)
bin_size

0.02

In [25]:
component_pos['LIT101']

1

In [26]:

# add paraam dict here and pass
def run_trial(environment, data_array, window_size, episodes, q_table=None):
    '''
    Function to run agent with a given starting point of a data window from the data_set
    :param environment: SWaTEnv
    :param model_input_window: Random slice of SWaT data in format for LSTM (1,19,37)
    :param number_of_episodes: Times to repeat
    :return: 
    '''
    
    # Q learning hyper parameters 
    # learning rate
    alpha = 0.1  
    # Future discount factor
    gamma = 0.9    
    # Initial exploration rate 
    epsilon = 1.0    
    # reduce exploration at each episode
    epsilon_decay = 0.999
    # final exploration rate/ stochastic behaviour
    min_epsilon = 0.01
    # number of episodes
    episodes = episodes 
    
    # Discrete levels for continuous values, normalised data so range 0-1
    num_bins = 50  
    # get bin size, use 2 to allow for outside of normal range
    bin_size = (2/num_bins)
    
    
    #this_env = environment(data_array, window_size, episode_length)
    # env is reset after every episode so no internal counter
    this_env = environment(data_array, window_size)
    
    #intitalise with zeros if one not passed
    if q_table is None:
        # Initialise Q-table with zeros, dims: number of possible states * number of actions
        q_table = np.zeros((num_bins, this_env.action_space.n))
    
    # make sure array is 3 dims for model input
    if len(data_array.shape) == 2:
        # Add a batch dimension to make it (1, time_steps, features)
        data_array = data_array[np.newaxis, :, :]
        
        

    
    # store lit101 values form each episode
    #lit101_list = []
    
    # target variable
    target_loc = component_pos['LIT101']
    
    
    
    #episodes = episode_length
    for episode in range(1, episodes+1):
        
        this_env.reset(data_array)
        state = this_env.state
        # just to initialise
        n_state = this_env.state
        #print(n_state)
        
        state_idx = int(np.round((n_state[target_loc]/ bin_size),2))
        done = False
        score = 0 
        
        while not done:
            #env.render()
            # sample from 4 possible states 
            
            # compare epsilon to random number, as epsilon decreases chance of random exploration decreases 
            if np.random.uniform(0, 1) < epsilon:
                # random explore
                action = this_env.action_space.sample()
            else:
                # take best action for this state for q table
                # deliberate exploit
                action = np.argmax(q_table[state_idx]) 
                
            # force action    
            # just activate MV101 to see if response from lstm model is linear
            #action = 1
            
            # pass 1 x 37 array of SWat states
            #action = one_state
            #n_state, reward, done, info = env.step(action)
            n_state, reward, value_change, done = this_env.step(action)
            
            # bin new state of target variable
            target_state_idx = int(np.round((n_state[target_loc]// bin_size),2))
            
            # Q-learning update rule
            q_table[state_idx, action] = q_table[state_idx, action] + alpha * (
            reward + gamma * np.max(q_table[target_state_idx]) - q_table[state_idx, action]
            )
            
            #print(n_state)
            score+=reward
        #print('Episode:{} Score:{}'.format(episode, score))
        # add this episodes list of values
        lit101_list = this_env.lit101_value_record
        
        # reduce exploration factor
        epsilon = max(min_epsilon, epsilon * epsilon_decay)
    return lit101_list, q_table
    
    
    #return input_windows


In [63]:
# q table at each 2000 episodes
#q_table_list = []

In [27]:
#for _ in range(10):
#    lit101_li, q_table = run_trial(SwatEnv,data_array,model_window,2000)
#    q_table_list.append(q_table)

In [28]:
#q_table_list


In [29]:
#np.savez('/home/loz/Documents/GitHub/MSc-Project/final_code#/component_objs/q_table_list_2000s.npz', *q_table_list) 

# Create Deep Policy Table

# Create Deep Learning Model#

In [1]:
states = env.observation_space.shape
actions = env.action_space.n

NameError: name 'env' is not defined

In [24]:
states

(1, 37)

In [25]:
actions

4

In [26]:
def build_model(states, actions):
    model = Sequential()    
    model.add(Dense(24, activation='relu', input_shape=states))
    #model.add(Dense(24, activation='relu', input_shape=(1,)))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [27]:
#del model

In [28]:
model = build_model(states, actions)

/home/loz/Documents/GitHub/MSc-Project/venv/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [29]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 1, 24)          │           912 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1, 24)          │           600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1, 4)           │           100 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,612 (6.30 KB)

 Trainable params: 1,612 (6.30 KB)

 Non-trainable params: 0 (0.00 B)

# Build Agent wit Keras-RL

In [30]:
#@tf.function(reduce_retracing=True)
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy,
                  nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)
    return dqn

In [31]:
dqn = build_agent(model, actions)

NameError: name 'BoltzmannQPolicy' is not defined

In [41]:
dqn.compile(Adam(learning_rate=1e-3), metrics=['mae'])

In [42]:


dqn.fit(env, nb_steps=5000, visualize=False, verbose=1)


Training for 5000 steps ...
Interval 1 (0 steps performed)
 1423/10000 [===>..........................] - ETA: 1:36 - reward: -0.5264done, took 16.064 seconds


In [23]:
scores = dqn.test(env, nb_episodes=100, visualize=False)
print(np.mean(scores.history['episode_reward']))

Testing for 100 episodes ...
Episode 1: reward: 10.000, steps: 60
Episode 2: reward: 16.000, steps: 60
Episode 3: reward: 6.000, steps: 60
Episode 4: reward: 8.000, steps: 60
Episode 5: reward: 4.000, steps: 60
Episode 6: reward: 18.000, steps: 60
Episode 7: reward: -16.000, steps: 60
Episode 8: reward: -12.000, steps: 60
Episode 9: reward: -8.000, steps: 60
Episode 10: reward: -2.000, steps: 60
Episode 11: reward: -2.000, steps: 60
Episode 12: reward: 8.000, steps: 60
Episode 13: reward: -10.000, steps: 60
Episode 14: reward: 4.000, steps: 60
Episode 15: reward: 2.000, steps: 60
Episode 16: reward: 6.000, steps: 60
Episode 17: reward: -10.000, steps: 60
Episode 18: reward: 0.000, steps: 60
Episode 19: reward: 10.000, steps: 60
Episode 20: reward: -8.000, steps: 60
Episode 21: reward: -6.000, steps: 60
Episode 22: reward: 6.000, steps: 60
Episode 23: reward: 0.000, steps: 60
Episode 24: reward: -2.000, steps: 60
Episode 25: reward: 8.000, steps: 60
Episode 26: reward: 6.000, steps: 60


In [12]:
os.getcwd()

'/home/loz/Documents/GitHub/MSc-Project/final_code'

In [26]:
#os.listdir()

In [24]:
# get pre_trained model of SWaT
swat_lstm = tf.keras.models.load_model('swat_lstm_100s.keras')


OSError: Unable to synchronously open file (file signature not found)